
# パラメータサーバー

:label: `sec_parameterserver`

単一の GPU から複数の GPU、さらに複数の GPU を含む複数のサーバー (おそらくすべてが複数のラックやネットワーク スイッチに分散している) に移行するにつれて、分散および並列トレーニングのアルゴリズムはさらに洗練される必要があります。インターコネクトごとに帯域幅が大きく異なるため、詳細が重要です (たとえば、NVLink は適切な設定で 6 つのリンク全体で最大 100 GB/秒を提供できますが、PCIe 4.0 (16 レーン) は 32 GB/秒を提供しますが、高速 100GbE イーサネットでもわずか10 GB/秒まで）。同時に、統計モデラーがネットワークとシステムの専門家であることを期待するのは不合理です。

パラメーター サーバーの中心的なアイデアは、分散型潜在変数モデルのコンテキストで :citet: `Smola.Narayanamurthy.2010`で紹介されました。プッシュおよびプルのセマンティクスの説明は :citet: `Ahmed.Aly.Gonzalez.ea.2012`に続き、システムとオープンソース ライブラリの説明は :citet: `Li.Andersen.Park.ea.2014`に続きます。以下では、効率化に必要なコンポーネントを動機付けします。

## データ並列トレーニング

分散トレーニングに対するデータ並列トレーニングのアプローチを確認してみましょう。実際に実装する方がはるかに簡単であるため、このセクションでは他のすべてを除外してこれを使用します。現在の GPU には十分なメモリがあるため、他の並列処理戦略が好まれるユースケースは (グラフでのディープ ラーニングを除けば) 事実上ありません。 :numref: `fig_parameterserver` :numref: `sec_multi_gpu`で実装したデータ並列処理のバリアントを説明します。この重要な点は、更新されたパラメーターがすべての GPU に再ブロードキャストされる前に、勾配の集約が 1 つの GPU (GPU 0) で行われることです。 

![](http://d2l.ai/_images/ps.svg) :label: `fig_parameterserver`

振り返ってみると、GPU 0 で集約するという決定はむしろ場当たり的なもののように思えます。結局のところ、CPU 上で集約した方がよいのかもしれません。実際、一部のパラメータを 1 つの GPU に集約し、他のパラメータを別の GPU に集約することもできます。最適化アルゴリズムがこれをサポートしているのであれば、サポートできない本当の理由はありません。たとえば、関連付けられた勾配 $\mathbf{g}_1, \ldots, \mathbf{g}_4$ を持つ 4 つのパラメーター ベクトルがある場合、$\mathbf{g}_i$ ごとに 1 つの GPU で勾配を集約できます ($ i = 1、\ldots、4$)。

この推論は恣意的で軽薄に思えます。結局のところ、数学は全体を通して同じです。ただし、 :numref: `sec_hardware`で説明したように、バスごとに帯域幅が異なる実際の物理ハードウェアを扱っています。 :numref: `fig_bw_hierarchy`で説明されている実際の 4 ウェイ GPU サーバーを考えてみましょう。特に接続が良好な場合は、100 GbE ネットワーク カードが搭載されている可能性があります。より一般的な数値は 1 ～ 10 GbE の範囲で、有効帯域幅は 100 MB/秒から 1 GB/秒です。すべての GPU に直接接続するには CPU の PCIe レーンが少なすぎるため (たとえば、コンシューマ グレードの Intel CPU には 24 レーンがあります)、[マルチプレクサ](https://www.broadcom.com/products/pcie-switches-bridges/pcie-switches)が必要です。 16x Gen3 リンク上の CPU からの帯域幅は 16 GB/秒です。これは、*各*GPU がスイッチに接続される速度でもあります。これは、デバイス間の通信がより効率的であることを意味します。

![](../img/bw-hierarchy.svg) :label: `fig_bw_hierarchy`

議論のために、勾配が 160 MB であると仮定しましょう。この場合、残りの 3 つの GPU すべてから 4 番目の GPU に勾配を送信するのに 30 ミリ秒かかります (各転送には 10 ミリ秒 = 160 MB / 16 GB/秒かかります)。重みベクトルを送信するためにさらに 30 ミリ秒を追加すると、合計 60 ミリ秒になります。すべてのデータを CPU に送信すると、4 つの GPU の*それぞれが*データを CPU に送信する必要があるため、40 ミリ秒のペナルティが発生し、合計で 80 ミリ秒かかります。最後に、勾配をそれぞれ 40 MB の 4 つの部分に分割できると仮定します。 PCIe スイッチはすべてのリンク間で全帯域幅の動作を提供するため、各部分を異なる GPU に*同時に*集約できるようになりました。 30 ミリ秒の代わりに 7.5 ミリ秒かかり、同期操作に合計 15 ミリ秒かかります。つまり、パラメータの同期方法に応じて、同じ操作に 15 ミリ秒から 80 ミリ秒かかる可能性があります。 :numref: `fig_ps_distributed`パラメーターを交換するためのさまざまな戦略を示しています。

![](../img/ps-distributed.svg) :label: `fig_ps_distributed`

パフォーマンスの向上に関しては、自由に使えるツールがさらにもう 1 つあることに注意してください。深いネットワークでは、上から下まですべての勾配を計算するのに時間がかかります。他のパラメーター グループの勾配の計算がまだ忙しい間でも、一部のパラメーター グループの勾配の同期を開始できます。 [Horovod](https://github.com/horovod/horovod)でこれを行う方法の詳細については、:citet: `Sergeev.Del-Balso.2018`を参照してください。

## リング同期

最新の深層学習ハードウェアでの同期に関しては、大幅に特注のネットワーク接続が必要になることがよくあります。たとえば、AWS p3.16xlarge インスタンスと NVIDIA DGX-2 インスタンスは、 :numref: `fig_nvlink`の接続構造を共有します。各 GPU は、最高 16 GB/秒で動作する PCIe リンクを介してホスト CPU に接続します。さらに、各 GPU には 6 つの NVLink 接続もあり、それぞれが 300 Gbit/s の双方向転送が可能です。これは、リンクごと、方向ごとに約 18 GB/秒に相当します。つまり、NVLink の合計帯域幅は PCIe 帯域幅よりも大幅に高くなります。問題は、それを最も効率的に使用する方法です。 

![](http://d2l.ai/_images/nvlink.svg) :ラベル: `fig_nvlink`

最適な同期戦略は、ネットワークを 2 つのリングに分解し、それらを使用してデータを直接同期することであることがわかりました (引用: `Wang.Li.Liberty.ea.2018` )。 :numref: `fig_nvlink_twoloop` 、ネットワークが 2 倍の NVLink 帯域幅を持つ 1 つのリング (1-2-3-4-5-6-7-8-1) と 1 つのリング (1-4-6-3-5) に分解できることを示しています。 -8-2-7-1)、通常の帯域幅。この場合、効率的な同期プロトコルを設計することは簡単ではありません。 

![](../img/nvlink-twoloop.svg) :label: `fig_nvlink_twoloop`

次の思考実験を考えてみましょう。$n$ のコンピューティング ノード (または GPU) のリングが与えられた場合、最初のノードから 2 番目のノードに勾配を送信できます。そこでローカル グラデーションに追加され、3 番目のノードに送信される、というように続きます。 $n-1$ ステップの後、最後にアクセスしたノードで集約勾配が見つかります。つまり、勾配を集約する時間はノード数に比例して増加します。しかし、これを行うとアルゴリズムは非常に非効率になります。結局のところ、通信しているノードは常に 1 つだけです。勾配を $n$ のチャンクに分割し、ノード $i$ から開始してチャンク $i$ の同期を開始したらどうなるでしょうか?各チャンクのサイズは $1/n$ であるため、合計時間は $(n-1)/n \約 1$ になります。言い換えれば、リングのサイズが大きくなっても、勾配の集約に費やされる時間は*長くなりません*。これはかなり驚くべき結果です。 :numref: `fig_ringsync` $n=4$ ノードでの一連のステップを示しています。 

![](http://d2l.ai/_images/ringsync.svg) :label: `fig_ringsync`

 8 つの V100 GPU 間で 160 MB を同期する同じ例を使用すると、約 $2 \cdot 160 \mathrm{MB} / (3 \cdot 18 \mathrm{GB/s}) \およそ 6 \mathrm{ms}$ になります。 。現在 8 つの GPU を使用していますが、これは PCIe バスを使用するよりも優れています。実際には、深層学習フレームワークが通信を大規模なバースト転送にまとめることができないことが多いため、これらの数値は少し悪いことに注意してください。

リング同期は他の同期アルゴリズムとは根本的に異なるというよくある誤解があることに注意してください。唯一の違いは、同期パスが単純なツリーと比較してやや複雑であることです。

## マルチマシントレーニング

複数のマシンでの分散トレーニングでは、さらなる課題が加わります。比較的低帯域幅のファブリックを介してのみ接続されているサーバーと通信する必要があり、場合によっては 1 桁以上遅くなる可能性があります。デバイス間での同期は難しいです。結局のところ、トレーニング コードを実行するさまざまなマシンの速度は微妙に異なります。したがって、同期分散最適化を使用したい場合は、それらを*同期する*必要があります。 :numref: `fig_ps_multimachine`分散並列トレーニングがどのように行われるかを示しています。
1. データの (異なる) バッチが各マシンで読み取られ、複数の GPU に分割されて GPU メモリに転送されます。そこでは、予測と勾配が各 GPU バッチで個別に計算されます。
1. すべてのローカル GPU からの勾配が 1 つの GPU に集約されます (または、その一部が異なる GPU にまたがって集約されます)。
1. 勾配は CPU に送信されます。
1.  CPU は、すべての勾配を集約する中央パラメータ サーバーに勾配を送信します。
1. 次に、集約された勾配を使用してパラメータが更新され、更新されたパラメータが個々の CPU にブロードキャストで戻されます。
1. 情報は 1 つ (または複数) の GPU に送信されます。
1. 更新されたパラメータはすべての GPU に分散されます。 

![](../img/ps-multimachine.svg) :label: `fig_ps_multimachine`

これらの操作はそれぞれかなり簡単に見えます。そして実際、それらは 1 台のマシン*内で*効率的に実行できます。しかし、複数のマシンを見てみると、中央のパラメータ サーバーがボトルネックになっていることがわかります。結局のところ、サーバーあたりの帯域幅は限られているため、$m$ ワーカーの場合、すべての勾配をサーバーに送信するのにかかる時間は $\mathcal{O}(m)$ になります。サーバーの数を $n$ に増やすことで、この障壁を突破できます。この時点で、各サーバーはパラメーターの $\mathcal{O}(1/n)$ を保存するだけで済みます。したがって、更新と最適化にかかる合計時間は $\mathcal{O}(m/n)$ になります。両方の数値を一致させると、処理するワーカーの数に関係なく、一定のスケーリングが得られます。実際には、*同じ*マシンをワーカーとしてもサーバーとしても使用します。 :numref: `fig_ps_multips`設計を示しています (詳細については、:cite: `Li.Andersen.Park.ea.2014`も参照してください)。特に、複数のマシンが不当な遅延なく動作することを保証することは簡単ではありません。 

![](../img/ps-multips.svg) :label: `fig_ps_multips`

## キーと値のストア

実際に分散マルチ GPU トレーニングに必要な手順を実装するのは簡単ではありません。このため、共通の抽象化、つまり再定義された更新セマンティクスを備えた*キーと値のストア*の抽象化を使用することが有益です。

多くのワーカーと多くの GPU にわたる勾配 $i$ の計算は次のように定義できます。

 $$\mathbf{g} *{i} = \sum* {k \in \text{労働者}} \sum_{j \in \text{GPU}} \mathbf{g}_{ijk},$$

ここで、$\mathbf{g}_{ijk}$ は、ワーカー $k$ の GPU $j$ で分割された勾配 $i$ の一部です。この演算の重要な点は、それが*可換的リダクション*であること、つまり、多くのベクトルを 1 つに変換し、演算が適用される順序は重要ではないということです。どの勾配をいつ受信するかを細かく制御する必要がない（必要がない）ため、これは私たちの目的にとっては最適です。また、この操作は異なる $i$ 間で独立していることに注意してください。

これにより、次の 2 つの操作を定義できるようになります: 勾配を蓄積する*Push*と、集計された勾配を取得する*Pull*です。さまざまなグラデーションのセットが多数あるため (結局のところ、レイヤーが多数あるため)、キー $i$ を使用してグラデーションにインデックスを付ける必要があります。 Dynamo :cite: `DeCandia.Hastorun.Jampani.ea.2007`で導入されたもののようなキーバリュー ストアとのこの類似性は偶然ではありません。これらも、特に複数のサーバーにパラメータを分散する場合に、同様の多くの特性を満たします。

キー/値ストアのプッシュ操作とプル操作は次のように説明されます。
-  **Push(key, value) は、**特定の勾配 (値) をワーカーから共通ストレージに送信します。そこで、値は、例えば合計することによって集約される。
-  **pull(key, value) は、**たとえばすべてのワーカーからの勾配を結合した後、共通ストレージから集計値を取得します。

同期に関するすべての複雑さを単純なプッシュおよびプル操作の背後に隠すことで、最適化を簡単な言葉で表現できるようにしたい統計モデラーと、分散同期に固有の複雑さに対処する必要があるシステム エンジニアの懸念を切り離すことができます。

## まとめ
- 同期は、特定のネットワーク インフラストラクチャとサーバー内の接続に高度に適応する必要があります。これにより、同期にかかる時間に大きな違いが生じる可能性があります。
- リング同期は、p3 および DGX-2 サーバーに最適です。他の人にとっては、それほどではないかもしれません。
- 階層型同期戦略は、帯域幅を増やすために複数のパラメータ サーバーを追加する場合にうまく機能します。

## 演習
1. リング同期をさらに高めることはできますか?ヒント: メッセージは双方向に送信できます。
1. 非同期通信(計算途中)を許可することは可能ですか?それはパフォーマンスにどう影響しますか?
1. 長時間の計算中にサーバーが失われた場合はどうなるでしょうか?計算を完全に再開しないようにするには、*フォールト トレランス*メカニズムをどのように設計すればよいでしょうか?

[ディスカッション](https://discuss.d2l.ai/t/366)
